Customer_Behaviour_Analysis

In [12]:
import pandas as pd
import mysql.connector as db
#connecting to db
db_connection=db.connect(
        user = 'Pavi',
        password = 'Pavi2001!',
        host = 'localhost',
        database = 'shopeasy',
        autocommit = True
)
db_connection   
curr = db_connection.cursor()

In [14]:

#read customers csv
customers_csv = pd.read_csv("customers.csv")
#create table
curr.execute(
    '''
    create table if not exists customers (CustomerID int, CustomerName varchar(30), Email varchar(30), Gender varchar(10), Age int, GeographyID int)
    ''')
#data insertion
insert_rows = ''' insert into customers (CustomerID,CustomerName,Email,Gender,Age,GeographyID) values (%s,%s,%s,%s,%s,%s)'''
rows_values = [tuple(x) for x in customers_csv.to_numpy()]
#data execution
curr.executemany(insert_rows,rows_values)
db_connection.commit()

In [15]:
#product csv
products_csv = pd.read_csv("products.csv")
curr.execute(
    '''
    create table if not exists products (ProductID int,ProductName varchar(30),Category varchar(10),Price int)
    '''
)
insert_products_row = ''' insert into products (ProductID,ProductName,Category,Price) values (%s,%s,%s,%s)'''
rows_values_products =[tuple(x) for x in products_csv.to_numpy()]
curr.executemany(insert_products_row,rows_values_products)
db_connection.commit()

In [18]:
#geography
geography_csv = pd.read_csv("geography.csv")
curr.execute(
    '''
    create table if not exists geography (GeographyID int,Country varchar(30),City varchar(30))
    '''
)
insert_geography_rows = '''
insert into geography (GeographyID,Country,City) values (%s,%s,%s)
'''
insert_row_values = [tuple(x) for x in geography_csv.to_numpy()]
curr.executemany(insert_geography_rows,insert_row_values)
db_connection.commit()

In [20]:
#customer_reviews
customer_reviews = pd.read_csv("customer_reviews.csv")
customer_reviews['ReviewDate'] = pd.to_datetime(customer_reviews['ReviewDate'])

curr.execute(
    ''' 
    create table if not exists customer_reviews (ReviewID int,CustomerID int, ProductID int,ReviewDate date,Rating int,ReviewText varchar(65) )
    '''
)
#column size is too long use alter to modify
curr.execute(
    '''
    alter table customer_reviews modify ReviewText varchar(5000)
    '''
) 

insert_customer_reviews = '''
insert into customer_reviews (ReviewID, CustomerID, ProductID,ReviewDate, Rating, ReviewText) values (%s,%s,%s,%s,%s,%s)
'''
insert_values=[tuple(x) for x in customer_reviews.to_numpy()]
curr.executemany(insert_customer_reviews,insert_values)
db_connection.commit()

In [22]:
#customer_journey
customer_journey = pd.read_csv("customer_journey.csv")

#change it to date format'pd.to_datetime()'
customer_journey['VisitDate'] = pd.to_datetime(customer_journey['VisitDate'])

#replace empty cells with '0'
customer_journey['Duration']= customer_journey['Duration'].fillna(value = 0)
customer_journey['Stage'] = customer_journey['Stage'].astype('str').str.title()

curr.execute(
    '''
    create table if not exists customer_journey(JourneyID int, CustomerID int,ProductID int,VisitDate date, Stage varchar(20),
    Action varchar(30), Duration float)
    '''
)

insert_journey = ''' 
insert into customer_journey(JourneyID, CustomerID,ProductID ,VisitDate, Stage, Action, Duration) values (%s,%s,%s,%s,%s,%s,%s)
'''

insert_values = [tuple(x) for x in customer_journey.to_numpy()]
curr.executemany(insert_journey,insert_values)
db_connection.commit()

In [24]:
#engagement_data
engagement = pd.read_csv("engagement_data.csv")

engagement['EngagementDate'] = pd.to_datetime(engagement['EngagementDate'])

curr.execute(
    '''
    create table if not exists engagement_data(EngagementID int,ContentID int,ContentType varchar(30),
    Likes int,EngagementDate date,CampaignID int, ProductID int, ViewsClicksCombined varchar(30))
    '''
)

insert_engagement ='''
insert into engagement_data(EngagementID ,ContentID ,ContentType,Likes,EngagementDate,CampaignID, ProductID, ViewsClicksCombined) 
values (%s,%s,%s,%s,%s,%s,%s,%s)
'''
insert_values  =[tuple(x) for x in engagement.to_numpy()]
curr.executemany(insert_engagement,insert_values)
db_connection.commit()

In [26]:
#factors influence customer engagement

data_blog = engagement[(engagement['ContentType']=='Socialmedia')|(engagement['ContentType']=='Newsletter')|(engagement['ContentType']=='Blog')|
    (engagement['ContentType']=='video')].groupby(
    ['ContentType','EngagementDate','Likes']).agg({
     'ViewsClicksCombined' : 'max'
}).reset_index()
result = data_blog.groupby("Likes").filter(lambda x:x["Likes"].max()>100)
result.groupby(['ContentType','EngagementDate','Likes']).agg({
    'ViewsClicksCombined' : 'max'
})


ViewsClicksCombined
ContentType EngagementDate Likes                    
Blog        2023-03-11     545             6205-2834
            2023-03-28     114              5280-532
            2023-07-23     840             6771-2617
            2023-08-30     190              1883-671
            2024-04-15     119              3511-479
Newsletter  2023-01-02     449              2674-956
            2023-10-29     118              1442-400
            2024-06-10     176              5147-788
Socialmedia 2023-04-26     142              1354-425
            2023-06-18     648             8237-1641
            2023-08-26     317             6837-2251
            2023-08-27     143              1671-482
            2024-06-01     193             4654-1083
video       2023-04-26     397             3632-1500
            2024-04-04     357             5543-1288

In [30]:
#what stage customer dropping
data_dropoff = customer_journey[(customer_journey["Action"] == "Drop-off") ].groupby(["Stage","Action"]).agg({
    'VisitDate' : ['max','min'],#check max,min
    'CustomerID' : ['count','max','min'],
    'Duration' : 'mean'
}).reset_index()

data_dropoff.columns = ['Action','Stage','VisitDate_max','VisitDate_min','Customer_count','Customer_max','Customer_min','Avg_duration'] #used to add columns
data_dropoff




,Action,Stage,VisitDate_max,VisitDate_min,Customer_count,Customer_max,Customer_min,Avg_duration
0,Checkout,Drop-off,2025-08-07,2023-07-13,14,94,1,0.0


In [32]:
# Reviews impact purchasing behaviour
data_impact = customer_reviews[(customer_reviews['Rating']<3)].groupby(['Rating','ReviewText']).agg({
    'ReviewDate': 'max'
})
data_impact

ReviewDate
Rating ReviewText                                                   
1      I  had  a  bad  experience  with  this  product.   2023-04-29
       Not  worth  the  money.                            2024-06-15
       The  product  stopped  working  after  a  month.   2023-01-07
2      Average  experience,  nothing  special.            2025-01-29
       Disappointed  with  the  performance.              2024-04-06
       Product  did    not  meet  my  expectations.       2025-01-13
       The  product    is  okay,  but  the  instructio... 2024-07-16
       The  product  arrived    late.                     2024-05-28

In [34]:
#product(productname),location(city),customer segmentation(Age,gender) performing well
#combine_cust_geo = pd.merge(customers_csv,geography_csv, on ='GeographyID', how = 'right')
#rename_product = products_csv.rename(column{'ProductID' : 'GeographyID'},inplace=True)
#loc_cust_perform = combine_cust_geo.groupby(['Gender', 'City']).agg({
 #   'Country': 'first',   # Assuming each city belongs to one country
  #  'Age': 'mean',        # Average age of customers in that segment
   # 'CustomerID': 'count' # Number of customers in that segment
#}).rename(columns={'Age': 'Avg_Age', 'CustomerID': 'Customer_Count'})
#loc_cust_perform

In [53]:
#Alterway
#product(productname),location(city),customer segmentation(Age,gender) performing well
combine_cust_geo = pd.merge(customers_csv,geography_csv, on ='GeographyID', how = 'right')
rename_product = pd.DataFrame(products_csv.rename(columns = {'ProductID' : 'GeographyID'}))
perform_data = pd.merge(combine_cust_geo, rename_product, on='GeographyID', how='right')
performance_factor = perform_data.groupby(['ProductName','City']).agg({
    'Country': 'first',   # Assuming each city belongs to one country
    'Age': 'mean'        # Average age of customers in that segment    
})
performance_factor['City %']=((performance_factor['Age']/ performance_factor['Age'].sum())*100) .round().fillna(0)
performance_factor


,,Country,Age,City %
ProductName,City,,,
Baseball Glove,Zurich,Switzerland,37.625000,9.0
Basketball,Brussels,Belgium,44.333333,11.0
Dumbbells,Madrid,Spain,42.833333,10.0
Fitness Tracker,Berlin,Germany,47.818182,11.0
Football Helmet,Stockholm,Sweden,32.500000,8.0
Golf Clubs,Vienna,Austria,44.000000,10.0
Running Shoes,London,UK,45.200000,11.0
Soccer Ball,Rome,Italy,34.416667,8.0
Tennis Racket,Amsterdam,Netherlands,43.111111,10.0


In [39]:
#identify key complaints an improvement areas
#customer_journey merge review and engagement -- ProductID
#action,stage,duration from customer_journey
#customer_reviews--rating,reviewtext
#engagement--ViewsClicksCombined

complaint_data = pd.merge(customer_journey,customer_reviews, on ='ProductID', how='left')
improvement_data = pd.merge(complaint_data,engagement, on ='ProductID', how='outer')
Review_duplicate = improvement_data.drop_duplicates(subset=['ReviewText'], keep='first')
combined_data = Review_duplicate.groupby(['Rating','ViewsClicksCombined','ReviewText']).agg({
    'Action' : 'first',
    'Rating' : 'min'
})
data = combined_data[(combined_data['Rating']<=3)&(combined_data['Action']!='Drop-off')]
data

Action  \
Rating ViewsClicksCombined ReviewText                                                
1.0    1079-44             The  product  stopped  working  after  a  month.   View   
2.0    5116-1524           The  product  arrived    late.                     View   
       750-34              Product  did    not  meet  my  expectations.       View   
       7877-2037           Disappointed  with  the  performance.             Click   
3.0    1010-196            Not    worth  the  money.                          View   
       1079-44             Good  quality,  but  could  be    cheaper.         View   
                           I  love  this    product,  will  buy  again!       View   
       1883-671            Customer  support  was    very  helpful.           View   
       3225-337            Average  experience,  nothing    special.          View   
       750-34              Not  worth  the  money.                            View   
       9759-3095           Average  experience,  nothing  special.            View   

                                                                             Rating  
Rating ViewsClicksCombined ReviewText                                                
1.0    1079-44             The  product  stopped  working  after  a  month.     1.0  
2.0    5116-1524           The  product  arrived    late.                       2.0  
       750-34              Product  did    not  meet  my  expectations.         2.0  
       7877-2037           Disappointed  with  the  performance.                2.0  
3.0    1010-196            Not    worth  the  money.                            3.0  
       1079-44             Good  quality,  but  could  be    cheaper.           3.0  
                           I  love  this    product,  will  buy  again!         3.0  
       1883-671            Customer  support  was    very  helpful.             3.0  
       3225-337            Average  experience,  nothing    special.            3.0  
       750-34              Not  worth  the  money.                              3.0  
       9759-3095           Average  experience,  nothing  special.              3.0

In [42]:
#pattern b/w (-)ve review and perform review

review_data = pd.merge(customer_reviews,products_csv, on = "ProductID", how ='right')
fill_data = review_data.fillna(0)
data = fill_data.groupby(['Rating','ProductID','ProductName']).agg({
    'ReviewText' : 'max',
    'Rating' :'min'
})

pattern_data = data[(data['Rating']<3) & (data['Rating']!=0)]
pattern_data

ReviewText  \
Rating ProductID ProductName                                                          
1.0    3         Yoga Mat                                   Not  worth  the  money.   
       4         Dumbbells         The  product  stopped  working  after  a  month.   
       6         Tennis Racket     I  had  a  bad  experience  with  this  product.   
2.0    2         Fitness Tracker       Product  did    not  meet  my  expectations.   
       7         Basketball       The  product    is  okay,  but  the  instructi...   
       10        Golf Clubs                   Disappointed  with  the  performance.   
       15        Climbing Rope                       The  product  arrived    late.   
       16        Kayak                      Average  experience,  nothing  special.   
       17        Surfboard                  Average  experience,  nothing  special.   

                                  Rating  
Rating ProductID ProductName              
1.0    3         Yoga Mat            1.0  
       4         Dumbbells           1.0  
       6         Tennis Racket       1.0  
2.0    2         Fitness Tracker     2.0  
       7         Basketball          2.0  
       10        Golf Clubs          2.0  
       15        Climbing Rope       2.0  
       16        Kayak               2.0  
       17        Surfboard           2.0

In [44]:
#Customer Sentiment performance data

perform_data = fill_data.groupby(['ProductID','ProductName','Rating']).agg({
    'Rating':'first',
    'ReviewText' : 'max',
    
})
perform_data[(perform_data['Rating']>3) | (perform_data['Rating']<=3)].drop_duplicates(subset = "ReviewText", keep='first')


Rating  \
ProductID ProductName     Rating           
1         Running Shoes   3.0        3.0   
                          4.0        4.0   
                          5.0        5.0   
2         Fitness Tracker 2.0        2.0   
                          3.0        3.0   
                          4.0        4.0   
                          5.0        5.0   
4         Dumbbells       1.0        1.0   
                          5.0        5.0   
5         Soccer Ball     4.0        4.0   
6         Tennis Racket   1.0        1.0   
                          3.0        3.0   
                          5.0        5.0   
7         Basketball      2.0        2.0   
8         Football Helmet 5.0        5.0   
9         Baseball Glove  4.0        4.0   
10        Golf Clubs      2.0        2.0   
                          5.0        5.0   
11        Ski Boots       3.0        3.0   
                          4.0        4.0   
14        Cycling Helmet  0.0        0.0   
15        Climbing Rope   2.0        2.0   
                          4.0        4.0   
17        Surfboard       3.0        3.0   
                          4.0        4.0   
18        Volleyball      4.0        4.0   
19        Hockey Stick    4.0        4.0   

                                                                         ReviewText  
ProductID ProductName     Rating                                                     
1         Running Shoes   3.0               Average  experience,  nothing  special.  
                          4.0                          The  quality  is  top-notch.  
                          5.0                Customer  support  was  very  helpful.  
2         Fitness Tracker 2.0          Product  did    not  meet  my  expectations.  
                          3.0                               Not  worth  the  money.  
                          4.0     Shipping  was  fast  and  the  item  was  well...  
                          5.0             Five  stars    for  the  quick  delivery.  
4         Dumbbells       1.0      The  product  stopped  working  after  a  month.  
                          5.0                The    build  quality  is  impressive.  
5         Soccer Ball     4.0     Shipping  was  fast    and  the  item  was  we...  
6         Tennis Racket   1.0      I  had  a  bad  experience  with  this  product.  
                          3.0     The  product  is  okay,  but  the  instruction...  
                          5.0                    Great  purchase,  very  satisfied.  
7         Basketball      2.0     The  product    is  okay,  but  the  instructi...  
8         Football Helmet 5.0                        The  quality    is  top-notch.  
9         Baseball Glove  4.0                  The  build  quality  is  impressive.  
10        Golf Clubs      2.0                 Disappointed  with  the  performance.  
                          5.0               Five  stars  for  the  quick  delivery.  
11        Ski Boots       3.0              Good  quality,  but  could  be  cheaper.  
                          4.0                        The  quality  is    top-notch.  
14        Cycling Helmet  0.0                                                     0  
15        Climbing Rope   2.0                        The  product  arrived    late.  
                          4.0                         Exceeded  my    expectations!  
17        Surfboard       3.0                             Not    worth  the  money.  
                          4.0     Shipping  was  fast  and  the  item    was  we...  
18        Volleyball      4.0             Five  stars  for    the  quick  delivery.  
19        Hockey Stick    4.0                  Great  purchase,    very  satisfied.

In [47]:
#conversion %
conversion_journey = pd.merge(customer_journey,customers_csv,on='CustomerID',how='right')
conversion_data = conversion_journey.groupby(['Action','Stage'])[['CustomerID']].nunique()#.reset_index()
conversion_data = conversion_data.sort_values(by='CustomerID', ascending=False)
conversion_data['Conversion_rate %'] = ((conversion_data['CustomerID'].shift(-1)/conversion_data['CustomerID']) * 100).round()
conversion_data['Drop-off %'] = (100 - conversion_data['Conversion_rate %'])#.mean()
conversion_data


CustomerID  Conversion_rate %  Drop-off %
Action   Stage                                                 
View     Homepage             32               59.0        41.0
         Productpage          19               84.0        16.0
Click    Homepage             16               88.0        12.0
Drop-off Checkout             14               43.0        57.0
Purchase Checkout              6               83.0        17.0
Click    Productpage           5                NaN         NaN

In [51]:
products_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    20 non-null     int64  
 1   ProductName  20 non-null     object 
 2   Category     20 non-null     object 
 3   Price        20 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 772.0+ bytes


In [172]:
filter_data = review_data[(review_data['Price']>200)]
highest_product = filter_data.groupby(['ProductName','Price'])[['Rating']].max().drop_duplicates().fillna(0).sort_values(by='Price' , ascending=False)
highest_product

,,Rating
ProductName,Price,
Cycling Helmet,472.32,0.0
Climbing Rope,410.17,5.0
Baseball Glove,327.36,4.0
Basketball,225.12,3.0


In [170]:
filter_data = review_data[(review_data['Price']<200)]
lowest_product = filter_data.groupby(['ProductName','Price'])[['Rating']].min().drop_duplicates().sort_values(by='Price')
lowest_product

,,Rating
ProductName,Price,
Dumbbells,26.21,1.0
Ice Skates,37.56,3.0
Football Helmet,44.75,5.0
Hockey Stick,173.83,4.0
Fitness Tracker,196.68,2.0
